In [ ]:
# ======================================================
# 🚀 NASA Space Apps - SAR Automation Pipeline (v7)
# Robust batch exports with diagnostics (Sentinel-1 GRD)
# ======================================================

import ee, geemap, requests, json, datetime, time, csv, os, shutil, zipfile, tempfile, geopandas as gpd
from google.colab import drive

# ---------------- 1) LOAD CONFIG ----------------
CONFIG_URL = "https://raw.githubusercontent.com/JuanoS12/Space-Apps/main/scripts/automate/config.json"

def load_config(url=CONFIG_URL):
    r = requests.get(url); r.raise_for_status()
    print("✅ Config loaded:", url)
    return r.json()

cfg = load_config()
PROJECT_ID   = cfg["project_id"]
BUCKET       = cfg["bucket"]
KMZ_PATH     = cfg["aoi_kmz"]
START_YEAR   = cfg["start_year"]
END_YEAR     = cfg["end_year"]
EXPORT_SCALE = cfg.get("export_scale_m", 10)
CRS          = cfg.get("crs", "EPSG:32616")
ORBIT        = cfg.get("orbit_pass", "ANY").upper().strip()  # "ASCENDING" | "DESCENDING" | "ANY"

# ---------------- 2) INIT EARTH ENGINE ----------------
ee.Authenticate()
ee.Initialize(project=PROJECT_ID)
print("✅ Earth Engine initialized.")
print(f"✅ Earth Engine project being used: {PROJECT_ID}")

# ---------------- 3) KMZ → EE FeatureCollection ----------------
# Moved definition before its usage in load_aoi_from_kmz
def download_and_extract_kml(kmz_url):
    local_kmz = "/content/aoi.kmz"
    print(f"\n⬇️ Downloading AOI from Drive:\n{kmz_url}")
    r = requests.get(kmz_url)
    if r.status_code != 200:
        raise ValueError(f"❌ Download failed ({r.status_code})")
    with open(local_kmz, "wb") as f: f.write(r.content)
    print(f"✅ KMZ downloaded: {local_kmz}")

    with zipfile.ZipFile(local_kmz, "r") as z:
        tmpdir = tempfile.mkdtemp()
        z.extractall(tmpdir)
        kmls = [os.path.join(tmpdir, f) for f in os.listdir(tmpdir) if f.endswith(".kml")]
        if not kmls: raise ValueError("❌ No .kml found inside KMZ!")
        print(f"✅ Extracted KML: {kmls[0]}")
        return kmls[0]

def load_aoi_from_kmz(kmz_url, buffer_width_m=5000):
    kml_path = download_and_extract_kml(kmz_url) # download_and_extract_kml is now defined
    gdf = gpd.read_file(kml_path, driver="KML")
    print(f"✅ GeoDataFrame loaded: {len(gdf)} feature(s).")

    # Buffer LineStrings in ~degrees
    polys = []
    for _, row in gdf.iterrows():
        geom = row.geometry
        if geom is None: continue
        if geom.geom_type == "LineString":
            buffer_deg = buffer_width_m / 111_320.0
            polys.append(geom.buffer(buffer_deg))
        elif geom.geom_type in ("Polygon","MultiPolygon"):
            polys.append(geom)
        else:
            print(f"⚠️ Skipping unsupported geometry: {geom.geom_type}")

    if not polys:
        raise ValueError("❌ No valid geometries after conversion.")

    poly_gdf = gpd.GeoDataFrame(geometry=polys, crs="EPSG:4326")
    geojson_path = "/content/aoi_polygons.geojson"
    poly_gdf.to_file(geojson_path, driver="GeoJSON")
    print(f"✅ AOIs saved to: {geojson_path}")

    fc = geemap.geojson_to_ee(geojson_path)
    print(f"✅ AOI converted to EE FeatureCollection ({fc.size().getInfo()} features).")
    return fc

lines_fc = load_aoi_from_kmz(KMZ_PATH)

# ---------------- 4) QUARTERS (end-exclusive) ----------------
def quarters_exclusive(start_year, end_year):
    Q = []
    for y in range(start_year, end_year + 1):
        Q.append((f"{y}-01-01", f"{y}-04-01", f"{y}_Q1"))
        Q.append((f"{y}-04-01", f"{y}-07-01", f"{y}_Q2"))
        Q.append((f"{y}-07-01", f"{y}-10-01", f"{y}_Q3"))
        Q.append((f"{y}-10-01", f"{y+1}-01-01", f"{y}_Q4"))
    return Q

quarters_list = quarters_exclusive(START_YEAR, END_YEAR)
print(f"\n🗓️ Generated {len(quarters_list)} quarters (end-exclusive).")

# ---------------- 5) Sentinel-1 base collection ----------------
def get_s1(geom, start, end, orbit_mode="ANY"):
    col = (ee.ImageCollection("COPERNICUS/S1_GRD")
           .filterBounds(geom)
           .filterDate(start, end)
           .filter(ee.Filter.eq("instrumentMode", "IW"))
           .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
           .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VH")))
    if orbit_mode in ("ASCENDING","DESCENDING"):
        col = col.filter(ee.Filter.eq("orbitProperties_pass", orbit_mode))
    return col

# ---------------- 6) Logging ----------------
os.makedirs("logs", exist_ok=True)
LOG_PATH = os.path.join("logs", "sar_exports_manifest.csv")
if not os.path.exists(LOG_PATH):
    with open(LOG_PATH, "w", newline="") as f:
        csv.writer(f).writerow(
            ["Section","Label","Start_Date","End_Date","Longitude","Latitude","File","Status","Count"]
        )

def log_row(section,label,s,e,lon,lat,fn,status,count):
    with open(LOG_PATH, "a", newline="") as f:
        csv.writer(f).writerow([section,label,s,e,lon,lat,fn,status,count])

# ---------------- 7) Export function ----------------
def export_sar(geom, start, end, label, name, orbit_mode="ANY"):
    col = get_s1(geom, start, end, orbit_mode)
    count = col.size().getInfo()
    print(f"   • {label}: {count} images")

    if count == 0:
        log_row(name,label,start,end,None,None,f"{name}_{label}.tif","NO_DATA",0)
        return False

    # Median of linear sigma0
    comp_lin = col.median().select(["VV","VH"])
    vv_db = comp_lin.select("VV").log10().multiply(10).rename("VV_dB")
    vh_db = comp_lin.select("VH").log10().multiply(10).rename("VH_dB")
    ratio = comp_lin.select("VV").divide(comp_lin.select("VH")).rename("VV_VH_ratio")

    export_img = (comp_lin.addBands([vv_db, vh_db, ratio])
                          .clip(geom)
                          .unmask(0))

    # Name safe prefix
    safe_name = str(name if name not in (None,"") else "AOI").replace(" ","_")
    desc = f"{safe_name}_{label}"
    file_name = f"{desc}.tif"

    # Start export to Cloud Storage
    task = ee.batch.Export.image.toCloudStorage(
        image=export_img,
        description=desc,
        bucket=BUCKET,  # Use the bucket from config
        fileNamePrefix=f"CLEAN/Interest/{desc}", # Specify a folder within the bucket with leading slash
        region=geom,           # use exact geom
        scale=EXPORT_SCALE,
        crs=CRS,
        maxPixels=1e13
    )
    task.start()
    print(f"   📤 Started export: {desc}")
    # log centroid (best-effort)
    try:
        lon, lat = geom.centroid(1).coordinates().getInfo()
    except Exception:
        lon, lat = None, None
    log_row(safe_name,label,start,end,lon,lat,file_name,"STARTED",count)
    return True

# ---------------- 8) Prepare AOI names robustly ----------------
fc_size = lines_fc.size().getInfo()
fc_list = lines_fc.toList(fc_size)

def feature_name(f, idx):
    # try common property keys
    props = ee.Feature(f).toDictionary().getInfo()
    for key in ("name","Name","NAME","description","Description"):
        if key in props and props[key]:
            return str(props[key])
    return f"AOI_{idx+1}"

aoi_items = []
for i in range(fc_size):
    f = fc_list.get(i)
    nm = feature_name(f, i)
    aoi_items.append((nm, ee.Feature(f).geometry()))
print(f"\n📍 AOIs detected: {[nm for nm,_ in aoi_items]}")
print(f"🎯 ORBIT filter: {ORBIT}")

# ---------------- 9) Run exports ----------------
print("\n🧭 Starting SAR extraction process...")
started = 0; skipped = 0
for idx, (nm, gm) in enumerate(aoi_items, 1):
    print(f"\n🗺️ Processing {nm} ({idx}/{len(aoi_items)})")
    for (s, e, lbl) in quarters_list:
        ok = export_sar(gm, s, e, lbl, nm, ORBIT)
        started += int(ok); skipped += int(not ok)
        time.sleep(0.5)  # be gentle with task creation

print(f"\n✅ Launch summary: STARTED={started} | NO_DATA={skipped}")

# ---------------- 10) Push manifest to Drive ----------------
drive.mount('/content/drive')
drive_folder = "/content/drive/MyDrive/SAR_Exports"
os.makedirs(drive_folder, exist_ok=True)
dest_path = os.path.join(drive_folder, "sar_exports_manifest.csv")
shutil.copy(LOG_PATH, dest_path)
print(f"✅ Manifest uploaded: {dest_path}")
print("✅ All applicable exports launched. Check the Earth Engine Tasks tab.")

✅ Config loaded: https://raw.githubusercontent.com/JuanoS12/Space-Apps/main/scripts/automate/config.json
✅ Earth Engine initialized.
✅ Earth Engine project being used: nasa-space-apps-473722

⬇️ Downloading AOI from Drive:
https://drive.google.com/uc?export=download&id=1oPVRvGny1y-pDE-5G-ayTUpSGHw0Rlh-
✅ KMZ downloaded: /content/aoi.kmz
✅ Extracted KML: /tmp/tmpf7emi8s8/doc.kml
✅ GeoDataFrame loaded: 4 feature(s).
✅ AOIs saved to: /content/aoi_polygons.geojson
✅ AOI converted to EE FeatureCollection (4 features).

🗓️ Generated 24 quarters (end-exclusive).

📍 AOIs detected: ['AOI_1', 'AOI_2', 'AOI_3', 'AOI_4']
🎯 ORBIT filter: ANY

🧭 Starting SAR extraction process...

🗺️ Processing AOI_1 (1/4)
   • 2020_Q1: 7 images
   📤 Started export: AOI_1_2020_Q1
   • 2020_Q2: 8 images
   📤 Started export: AOI_1_2020_Q2
   • 2020_Q3: 7 images
   📤 Started export: AOI_1_2020_Q3
   • 2020_Q4: 10 images
   📤 Started export: AOI_1_2020_Q4
   • 2021_Q1: 8 images
   📤 Started export: AOI_1_2021_Q1
   • 20